In [1]:
import pandas as pd
import numpy as np
import glob

import xml.etree.ElementTree as ET

In [2]:
def dataset_xml_reader(routes, dataset):
    quest_ans=[]

    for i_dtst,path in enumerate(routes):
        files = glob.glob(path)
        for name in files:
            tree = ET.parse(name)
            root = tree.getroot()
            dtset=dataset[i_dtst]
            quest_id=root.attrib["id"]
            question=root[0].text
            try:
                qtype=root.attrib["qtype"]
            except:
                qtype=None
            try:
                stype=root.attrib["stype"]
            except:
                stype=None

            grade_st=[branch.attrib["accuracy"] for branch in root[2]]
            id_st=[branch.attrib["id"] for branch in root[2]]
            answers_st=[branch.text for branch in root[2]]
            answers_ref=[branch.text for branch in root[1]]# If there are more of 1 reference answer it will give an array of an answers. Make sure you integrate it before processing
            for ith_ans,st_answ in enumerate(answers_st):
                for ith_ref_ans,ref_ans in enumerate(answers_ref):
                    st_ans_calif=(grade_st[ith_ans]=="correct")*1
                    quest_ans.append([dtset,qtype,stype,quest_id,question,ref_ans,st_answ,id_st[ith_ans],st_ans_calif])

    return(quest_ans)

# Trainning Dataset

In [3]:
routes_train=["dataset/training/2way/sciEntsBank/*.xml","dataset/training/2way/beetle/*.xml"]
dataset_train=["TRAIN SciEntsBank","TRAIN Beetle"]

In [4]:
train_quest_ans=dataset_xml_reader(routes_train,dataset_train)

In [5]:
vars_table=["dtset","qtype","stype","quest_id","question","ref_ans","st_answ","id_st","st_ans_calif"]
answers=pd.DataFrame(train_quest_ans, columns=vars_table)
answers.shape

(22167, 9)

In [6]:
answers.to_csv("train_answers.csv", sep=";")

# Evaluation Dataset

In [7]:
test_routes=["dataset/test/2way/sciEntsBank/test-unseen-answers/*.xml",
             "dataset/test/2way/sciEntsBank/test-unseen-domains/*.xml",
             "dataset/test/2way/sciEntsBank/test-unseen-questions/*.xml",
             "dataset/test/2way/beetle/test-unseen-answers/*.xml",
             "dataset/test/2way/beetle/test-unseen-questions/*.xml"]
test_datasets=["TEST sciEntsBank Unseen Answers",
               "TEST sciEntsBank Unseen Domains",
               "TEST sciEntsBank Unseen Questions",
               "TEST Beetle Unseen Answers",
               "TEST Beetle Unseen Questions"]

In [8]:
test_quest_ans=dataset_xml_reader(test_routes,test_datasets)

In [9]:
test_answers=pd.DataFrame(test_quest_ans, columns=vars_table)
test_answers.shape

(11596, 9)

In [10]:
test_answers.to_csv("test_answers.csv", sep=";")